<a href="https://colab.research.google.com/github/HiuNgaiChan/3dSinGAN/blob/master/FpWGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from scipy.stats import wasserstein_distance



dataCSV = pd.read_csv('noNorm1000.csv')
dataTensor = torch.tensor(dataCSV.values).to(torch.float32)
dataTensor = torch.flatten(dataTensor)
dataTensor = dataTensor.reshape(1,dataTensor.size()[0])
batch_size = 1
dataset = torch.utils.data.DataLoader(dataTensor, batch_size=batch_size, shuffle=True)


learningRate = 0.0001

input_dim = dataTensor.size()[1]
z_size = 1000



In [4]:
class DNet(nn.Module):
    def __init__(self):
      super().__init__()
      self.net = nn.Sequential(
          nn.Linear(input_dim,512),
          nn.ReLU(),
          #nn.Dropout(p=0.2),
          nn.Linear(512, 256),
          nn.ReLU(),
          #nn.Dropout(p=0.2),
          nn.Linear(256, 128),
          nn.ReLU(),
          #nn.Dropout(p=0.2),
          nn.Linear(128, 1),
      )


    def forward(self, x):

      return self.net(x)

class GNet(nn.Module):
    def __init__(self):
      super().__init__()
      self.net = nn.Sequential(
          nn.Linear(z_size, 128),
          nn.ReLU(),
          nn.Linear(128, 256),
          nn.ReLU(),
          nn.Linear(256,512),
          nn.ReLU(),
          nn.Linear(512,input_dim),
          nn.Tanh()
      )
    def forward(self, x):

      return self.net(x)

In [5]:
d = DNet()
g = GNet()

optimizerD = optim.RMSprop(d.parameters(), lr=0.00005)
optimizerG = optim.RMSprop(g.parameters(), lr=0.00005)

In [6]:
print("start training")

emdList = []
ksteps = 5
num_epochs = 7000
#z = torch.randn(batch_size,z_size)
for epochs in range(num_epochs):
  for batch in dataset:
    d_loss_real_list = torch.zeros(ksteps)
    d_loss_fake_list = torch.zeros(ksteps)
    for k in range(ksteps):
      #train the discriminator
      d.zero_grad()
      real_output = d(batch.view(batch.size()[0],input_dim))

      z = torch.randn(batch.size()[0],z_size)
      g_fake = g(z)
      fake_output = d(g_fake)

      d_loss_real_list[k] = real_output
      d_loss_fake_list[k] = fake_output


    d_loss = -torch.mean(d_loss_real_list) + torch.mean(d_loss_fake_list)
    d_loss.backward()
    optimizerD.step()
    for p in d.parameters():
      p.data.clamp_(-0.01, 0.01)

    #train the generator
    g.zero_grad()
    z = torch.randn(batch.size()[0],z_size)
    gd_output = d(g(z))
    g_loss = - gd_output
    g_loss.backward()
    optimizerG.step()

  if epochs%100 == 0:
    #print("Epochs[", epochs,"]: ", " d_loss -> ",round(d_loss.item(),2), " g_loss -> ", round(g_loss.item(),2))
    with torch.no_grad():
      z = torch.randn(1,z_size)
      g_output = g(z)
      emd = wasserstein_distance(dataTensor[0].tolist(), g_output[0].tolist())     
      print("Epochs[", epochs, "]: EMD = " + str(emd))
      emdList.append(emd)

start training
Epochs[ 0 ]: EMD = 0.49892120280422386
Epochs[ 100 ]: EMD = 0.04281176131440831
Epochs[ 200 ]: EMD = 0.03989829343947349
Epochs[ 300 ]: EMD = 0.024720790010488904
Epochs[ 400 ]: EMD = 0.019210291052479685
Epochs[ 500 ]: EMD = 0.04352921189802388
Epochs[ 600 ]: EMD = 0.0639309419907125
Epochs[ 700 ]: EMD = 0.019962486469419675
Epochs[ 800 ]: EMD = 0.04377200520172482
Epochs[ 900 ]: EMD = 0.03361344481060708
Epochs[ 1000 ]: EMD = 0.0361663229445985
Epochs[ 1100 ]: EMD = 0.02095512680047735
Epochs[ 1200 ]: EMD = 0.019459113801839217
Epochs[ 1300 ]: EMD = 0.02988796167051381
Epochs[ 1400 ]: EMD = 0.04643376954987374
Epochs[ 1500 ]: EMD = 0.015712671056651743
Epochs[ 1600 ]: EMD = 0.020568865983174568
Epochs[ 1700 ]: EMD = 0.012955450077318046
Epochs[ 1800 ]: EMD = 0.013227499535210276
Epochs[ 1900 ]: EMD = 0.021584253813246808
Epochs[ 2000 ]: EMD = 0.02788014745644371
Epochs[ 2100 ]: EMD = 0.02062667217265698
Epochs[ 2200 ]: EMD = 0.031806161620896695
Epochs[ 2300 ]: EMD = 0

In [1]:
xis = np.arange(0,num_epochs,100)
plt.plot(xis,emdList,'b-')
plt.ylabel('EMD')
plt.xlabel('Epochs')
plt.title('FPGAN Wasserstein 5k Cube EMD')
plt.grid(True)
plt.savefig("fpganWasserstein5kCubeEMD.png")
plt.show()

NameError: ignored

In [ ]:
#Checking if the generated point clouds are all unqiue to each other
from metrics import checkUnique

with torch.no_grad():
  record = np.zeros(10)
  for j in range(100):
    print(j)
    for i in range(10):
      z = torch.randn(1000*(i+1),z_size)
      g_output = g(z)
      g_output = g_output.numpy()
      #print(str(i+1) + " " + str(checkUnique(g_output)))
      if checkUnique(g_output) == True:
        record[i] += 1
print(record)

In [11]:
#Generating 10 point cloud csv files
import csv
import torch

numModelPts = int(dataTensor.size()[1]/3)

for i in range(10):
  filename = "Wasserstein5kPoint" + str(i) +".csv"
  with open(filename, "w", newline="") as csvfile:
    xyzWriter = csv.writer(csvfile, delimiter=",")
    with torch.no_grad():
      z = torch.randn(1,z_size)
      g_output = g(z)
      g_output = g_output.reshape(numModelPts,3).tolist()
      for i in range(1000):
        line = g_output[i]
        xyzWriter.writerow(line)

In [10]:
#Generates n number of cloud points and combine them into a single csv file
import csv
import torch
numModelPts = int(dataTensor.size()[1]/3)
num_cubes = 1000
filename = "1kwganCombinedCubes" + str(num_cubes) +".csv"
with open(filename, "w", newline="") as csvfile:
  xyzWriter = csv.writer(csvfile, delimiter=",")
  with torch.no_grad():
    for itr in range(num_cubes):
      z = torch.randn(1,z_size)
      g_output = g(z)
      g_output = g_output.reshape(numModelPts,3).tolist()
      for i in range(1000):
        line = g_output[i]
        xyzWriter.writerow(line)

In [ ]:
with torch.no_grad():
    z = torch.randn(1,z_size)
    g_output = g(z)
    print(g_output[0][0:5])
    print(dataTensor[0][0:5])

tensor([0.3248, 0.9390, 0.9971, 0.5473, 0.7481])
tensor([0.3591, 0.9180, 1.0000, 0.5380, 0.7089])
